In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# get file from here
# https://data.gov.ua/dataset/06779371-308f-42d7-895e-5a39833375f0
file_path = r"D:\IT\Pet_project_car\tz_opendata_z01012023_po01012024.csv"
df = pd.read_csv(file_path, nrows=2)
print(df)

In [ ]:
df = pd.read_csv(r'D:\IT\Pet_project_car\tz_opendata_z01012023_po01012024.csv', sep=';', low_memory=False)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['PERSON'].value_counts().head(10)

In [ ]:
df_prof=df[['PERSON', 'OPER_NAME', 'D_REG', 'BRAND', 'MODEL', 'MAKE_YEAR', 'COLOR', 'KIND', 'BODY', 'FUEL', 'CAPACITY']]

In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(df_prof)

In [ ]:
# json file is here https://data.gov.ua/dataset/06779371-308f-42d7-895e-5a39833375f0
# Create a new folder 'files' if it doesn't exist
if not os.path.exists('files'):
    os.makedirs('files')

with open('datapackage.json', 'r') as json_file:
    data = json.load(json_file)

for resource in data['resources']:
    path = resource['path']
    file_name = path.split("/")[-1]
    
    response = requests.get(path)
    
    if response.status_code == 200:
        #Save the file in the 'files' folder
        with open(os.path.join('files', file_name), 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {file_name}")

print("Download complete.")

Unzip and add to df

In [ ]:
# List files in the 'files' folder
files = os.listdir('files')

# Iterate through the files and unzip any zip archives
for file_name in files:
    if file_name.endswith('.zip'):
        # Specify the path of the zip file
        zip_path = os.path.join('files', file_name)

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Extract files into the 'files' folder
            zip_ref.extractall('files')
            print(f"Extracted: {file_name}")

        # Delete the zip file after extraction
        os.remove(zip_path)
        print(f"Deleted: {file_name}")

print("Extraction of zip files complete.")

In [ ]:
# Check files in the 'files' folder
folder_path = os.getcwd()
print("\nFiles in the 'files' folder:")
files_folder_path = os.path.join(folder_path, 'files')
for filename in os.listdir(files_folder_path):
    print(filename)

In [ ]:
# change '.ßsv' file
for filename in os.listdir(files_folder_path):
    if not filename.endswith('.csv'):
        # Rename the file to end with '.csv'
        new_filename = os.path.splitext(filename)[0] + '.csv'
        os.rename(os.path.join(files_folder_path, filename), os.path.join(files_folder_path, new_filename))
        print('file', filename, 'renamed')

In [ ]:
#Creat a df for all the data

result_df = pd.DataFrame()

for filename in os.listdir(files_folder_path):

    if filename.endswith('.csv'):
        file_path = os.path.join(files_folder_path, filename)
        df = pd.read_csv(file_path, sep = ';', low_memory=False)
        df.columns = df.columns.str.upper()
        result_df = pd.concat([result_df, df], ignore_index=True)
        print(filename, 'added')
print('I\'m done')

In [ ]:
print(len(result_df))

Filter df
фізичні особи,
первинна реєстрація
легкових автомобілів
загального призначення.

In [ ]:
#Optional: choose only relevant fields.  
result_df = result_df[['PERSON', 'OPER_CODE', 'OPER_NAME', 'D_REG', 'BRAND', 
                       'MODEL', 'MAKE_YEAR', 'COLOR', 'KIND', 'BODY',
                      'PURPOSE', 'FUEL', 'CAPACITY', 'VIN']]

#filter df 
result_df = result_df[(result_df['PERSON']=='P')&
                      (result_df['KIND'] =='ЛЕГКОВИЙ')&
                      (result_df['PURPOSE'] =='ЗАГАЛЬНИЙ')]

In [ ]:
#find necessary codes 
result_df[result_df['OPER_NAME'].str.upper().str.contains('ПЕРВИННА')]['OPER_CODE'].unique()

In [ ]:
#check the most popular codes, to find out 70 and 71 could also be relevand. I can be wrong =) 
result_df[['OPER_CODE', 'OPER_NAME']].value_counts().head(20) 

In [ ]:
#filter df to contain only ПЕРВИННА РЕЄСТРАЦІЯ (and 70, 71 )
list_of_codes = [105, 100,  99,  30, 172, 70, 71]
filtered_df = result_df[result_df['OPER_CODE'].isin(list_of_codes)]
filtered_df.reset_index(inplace=True)

In [ ]:
len(filtered_df) 

In [ ]:
#check if unique
filtered_df['VIN'].duplicated().sum() 

In [ ]:
duplicates = filtered_df['VIN'].duplicated()
filtered_df[duplicates].tail(6)

In [ ]:
#without NAs 
filtered_df[~filtered_df['VIN'].isna()].duplicated().sum()

In [ ]:
#only the columns that we are interested in 
firsts_df = filtered_df[['D_REG','BRAND', 'MODEL', 'MAKE_YEAR','COLOR','FUEL', 'CAPACITY']]
firsts_df.to_csv('first_reg_all_years.csv')
firsts_df.head()

In [ ]:
firsts_df.dtypes

Data format check

In [ ]:
years = [x+1 for x in range(2012,2023)]
for year in years:
    s = firsts_df[firsts_df['D_REG'].str.contains(str(year))]['D_REG']
    print(s.max())

In [ ]:
firsts_df[firsts_df['D_REG'].str.endswith('23')]['D_REG'].head(1)

In [ ]:
#converting all the dates. It takes a while. 
def convert_date(date_str):
    if '-' in date_str:
        # Dates from 2013 to 2018 are in the format '2013-12-30'
        return pd.to_datetime(date_str, format='%Y-%m-%d')
    elif len(date_str) == 10:
        # Dates from 2019 to 2022 are in the format '31.10.2019'
        return pd.to_datetime(date_str, format='%d.%m.%Y')
    else:
        # Dates in 2023 are in the format '20.10.23'
        return pd.to_datetime(date_str, format='%d.%m.%y')

# Apply the function to the 'D_REG' column
firsts_df['D_REG'] = firsts_df['D_REG'].apply(convert_date)

In [ ]:
firsts_df.dtypes

In [ ]:
frsts = firsts_df.copy()
frsts.head()

In [ ]:
frsts.loc[:, 'YEAR'] = frsts['D_REG'].dt.year

In [ ]:
# Group by 'YEAR' and count the number of rows for each year
car_count_by_year = frsts.groupby('YEAR').size()

# If you want the result as a DataFrame
car_count_by_year_df = pd.DataFrame(car_count_by_year, columns=['Car Count'])

In [ ]:
# trend of new cars 
sns.lineplot(x='YEAR', y='Car Count', marker='o', linestyle='-', color='b', data=car_count_by_year_df)
plt.title('Number of Cars First Registered Each Year')
plt.xlabel('Year')
plt.ylabel('Number of Cars')
plt.show()

In [ ]:
top_colors_by_year = frsts.groupby('YEAR')['COLOR'].value_counts().reset_index(name='Count')

pivoted_colors_by_year = top_colors_by_year.pivot(index='COLOR', columns='YEAR', values='Count')
pivoted_colors_by_year

In [ ]:
pivoted_colors_by_year.to_csv('colors_by_year.csv')

In [ ]:
frsts[frsts['YEAR']==2013][['BRAND', 'MODEL']].value_counts().head(10)
#this is bad

In [ ]:
#only the first word of 'wrong' brands will be a brand 
frsts.loc[:,'brand_only'] = frsts['BRAND'].apply(lambda x: x.split()[0])

In [ ]:
frsts['brand_only'].nunique()

In [ ]:
frsts['brand_only'].value_counts().tail(10)

In [ ]:
brands_by_year_all = frsts.groupby('YEAR')['brand_only'].value_counts().reset_index(name='Count')

# Rank the brands within each year 
brands_by_year_all.loc[:,'rank'] = brands_by_year_all.groupby('YEAR')['Count'].rank(method='dense', ascending=False)

# Filter out the top 10 brands for each year
top_brands_df = brands_by_year_all[brands_by_year_all['rank'] <= 10]

# Sorting by year and rank for better readability
top_brands_df = top_brands_df.sort_values(by=['YEAR', 'rank'])

# Reset index for the final DataFrame
top_brands_df.reset_index(drop=True, inplace=True)

In [ ]:
top_brands_df

In [ ]:
top_brands_df.to_csv('top_brands_count.csv')

In [ ]:
#same pivoting as with colors 
pivoted_brands_by_year_all = top_brands_df.pivot(index='brand_only', columns='YEAR', values='rank')

In [ ]:
#same pivoting as with colors
pivoted_brands_counts = top_brands_df.pivot(index='brand_only', columns='YEAR', values='Count')

In [ ]:
pivoted_brands_counts

In [ ]:
pivoted_brands_counts.to_csv('pivoted_brands_counts.csv')

In [ ]:
pivoted_brands_by_year_all

In [ ]:
pivoted_brands_by_year_all.to_csv('brands_by_year_all.csv')

In [ ]:
#same pivoting as with colors 
pivoted_brands_by_year_all = top_brands_df.pivot(index='brand_only', columns='YEAR', values='rank')


In [ ]:
#unpivoted table for heatmap
top_brands_df.to_csv('top_brands_heatmap.csv')

In [ ]:
fuel_by_year = frsts.groupby(['YEAR', 'FUEL']).size().reset_index(name='Count')

pivoted_fuel_by_year = fuel_by_year.pivot(index='FUEL', columns='YEAR', values='Count')

pivoted_fuel_by_year['electro'] = pivoted_fuel_by_year.index.str.contains('ЕЛЕКТРО')
electro = pivoted_fuel_by_year.groupby('electro').sum()
electro = electro.T
electro

In [ ]:
electro.to_csv('electro.csv')

In [ ]:
electro['year_total'] = electro[True] + electro[False]
electro['share_true'] = electro[True] / electro['year_total']
plt.bar(electro.index, electro['share_true'] * 100, color='blue', alpha=0.7)